# Job application for senior bioinformatician at Turbine

## Bulk RNA-Seq profiling of JQ1 treatment on SUM159 cell line

The project goal was to quantify the mRNA expression changes of JQ1 treatment in triple-negative breast cancer cell line (SUM159). This analysis has 4 parts, which will be descrbed in more details:
  * i) summary notebook (this one)
  * ii) data preprocessing notebook (bin/RNA_seq_data_preprocessing.ipynb)
  * iii) RNA quantfication from raw reads using SALMON and STAR (bin/RNA_seq_quantification.ipynb)
  * iv) statistical analysis and visualization with DESeq2 (bin/turbine_rna_deseq2.ipynb)



# Bulk RNA-Seq data processing
Next generation sequencing became routinly and easily acccessible for research groups resulting in the large-scale accumulating of sequencing information in databases. In the past few decades many tools have been proposed and it is not trivial to choose the 'best' tool combination or bioinformatic pipeline for a certain application. Fortunately, many efforts have been taken to support and build such pipelines, especially for RNA-Seq, which is one of the most commonly used technology for investigating the transcription profile of living organisms. Currently these pipeline tools (Workflow description language, WDL, CWL (Common Workflow language), Snakemake, Nextflow) (i.e. [1]) are the state-of-the-art approaches, because they not only help to build a robust pipeline, but they also support the scalibility (via abstractions), which is crucial in applications where the goal is to process 1000s or even more samples.
A good example is Snakemake [2] or Galaxy [3]; for a comprehensive review of the available sollutions see [4]. Here, on the other hand, I decided to go with a custom, POC pipeline for demonstration purposes, since this is a bioinformatics job application. The first part of the pipeline is done via python, the second via bash (i.e. genome indexing) and python, the third part (statistical evaluation) in R. 

## Infrastructure

All the calculations were carried out in cloud environment (Microsoft Azure and Google Cloud), where both traditional (IaaS) and serverless technologies (like cloud-run) were used. The input data is deposited in the cloud (links are in the notebooks). The software infrastructure (neccessary packages, tools, etc.) are provided via docker. All the codes, notebooks, docker files are deposited in a github repository. 
The containers are also available publicly and the exact versions of the softwares and libraries can be extracted. I belive this is important, otherwise the reproducibility is much harder to be guaranteed. 
Note, however, that these submitted codes, containers are for demonstration purposes only, not suitable for production usage (for example the containers can and should be optimized; security was not addressed etc.). There are many limitations in this version which can be elimated, at least partly, using proper, CWL like pipeline managers (i.e. collecting input paraemters). Ideally, for such a project one should not only use version control managers like github, but also proper CI/CD pipelines and integrations (like Jenkins).

## Methods (summary)
The general goal is to understand how the JQ1 treatment changes the transcriptiomic landscape of the cell. There are many ways to carry out an RNA-Seq analysis. First, one could choose from de-novo or 'reference' based methods. Personally, I was really interested whether using the recently published telomer-to-telomer reference genome (CHM13v2) [5] gives new insight about the underlying mechanism of JQ1 treatments. This new reference contains more then 2000 new genes (99 protein coding genes as well), which are likely to be relevant in cancer-related processes. However, the subsequent statistical analysis in DeSeq2 failed due to the proper support of annotation packages in R. The solution is compiling the neccessery mappings (i.e. from transcripts to genes, genes to GO categories, etc.). 

In this processing pipeline I used reference based tools, for example STAR [6] and SALMON [7]. The GRCh38.p13 reference genome was used withs its ensemble provided gene annotation. Note that the gene annotations have significant effect on the final outputs (i.e. significantly differential expressed genes). STAR is a traditional read mapping tool tailored to rna-seq like data (i.e. by providing splice aware alignments to reference genome), while SALMON is a k-mer (transcriptome) indexing based method with increasing popularity. 

There is also a wide variety of possible frameworks dealing with raw RNA-Seq data like edgeR [8], DESeq2 [9], etc. In this application I used DESeq2. This framework provides many functions for visualizing and analizing the rna-seq2 data. It provides a flexible statistical framework for modelling and comparing count data. Here I used a very-sample case-control description of the experiment. Note, however, that it could handle much more complex scenarios like the one presented in the original paper [10]. This experiment contains technical replicates of the treatments, which is recommended to be collapsed. For demonstration reasons, I skipped that step (better visualization and enrichment analysis).






# Background





Triple negative breast cancer is one of the most challanging types of breast cancer, since it’s associated with poor prognosis and overall survival and because of the limited therapeutic options. 

<img src="assets/41571_2021_565_Fig2_HTML.webp" style="height:1000px"/>

The TBNC associated signaling pathways and the potentially applicable therapeutic options, for a review see [12]. 






# Results

## PART 1 - Sample preprocessing

The quality of the raw reads were assesed with FastQC and Multiqc as presented in RNA_seq_data_preprocessing.ipynb notebook. Based on the assessment the first 13bp of reads were removed as well as the low quality parts. The reads with length of 30bp were discarded. The illumina adapters (True-seq) were removed using the trimmomatic 1.39. 
<h1>Summarized mutliqc report:</h1> <a href="https://rna-pipeline-4tzn6567yq-ey.a.run.app/files/results/raw_multiqc/multiqc_report.html"> MultiQC results </a> <p> (For details visit the documentation of multiQC <a href="https://multiqc.info"> https://multiqc.info </a>) </p> <br>

<h1>Summarized report after the preprocessing:</h1> <a href="https://rna-pipeline-4tzn6567yq-ey.a.run.app/files/results/trimmed_multiqc/multiqc_report.html"> MultiQC results </a> <p> (For details visit the documentation of multiQC <a href="https://multiqc.info"> https://multiqc.info </a>) </p> <br>

Later on both STAR and SALMON used the trimmed reads as input. For more details see bin/RNA_seq_data_preprocessing.ipynb.



## RNA quantification

The quantification process is documented in bin/RNA_seq_alignment_transcript_quant.ipynb notebook. 
The RNA quantification was done with either STAR (note that the index was fitted to this datasets) or SALMON. After the mapping process (for more details see the notebook) the alignment results are sorted with samtools. The STAR alignment results can be visualizing IGV. featureCounts were used to create the final count data table. 

The outputs are the following:
  * Salmon pseudo counts and statistic: results/salmon_GRCh38_outputs/*
  * STAR alignments and counts: results/trimmed_star_GRCh38/*

# Statistical analysis and visualization with DESeq2

Analyzing and interpreting RNA-Seq data is surprisingly complex due to various reasons. During the analysis many questions arise ranging from bioinformatics and biostatistics to biomedical ones and various data visualization techniques (i.e. clustered heatmaps, PCA) might help to address these questions. 
The RNA-Seq data is known to be compositional which has several implications regarding how to compare expression levels of the same genes across samples and groups or within sample. Here I applied different normalization and filtering techniques. 
The ideal outcome is a new biological knowledge i.e. better characterization of treatment. For demonstration purposes the technical replicates were not collapsed.


The main outputs:
  * normalized (and log-transformed) count data
  * PCA plot for comparing the samples
  * Differential expressed genes table: statistical analysis (significantly DE genes, p-value correction)
  * Clustered heatmaps
  * Gene set enrichment analyis (significantly affected KEGG pathways)
  * Other functional analysis (GO categories, basic pathways)


## Deseq2

#### Clustered heatmap
<img src="assets/salmon_GRCh38_samples_heatmap_gene_sample.png" style="height:500px"/>

This figure represents the association between the significantly (adjusted p-value) differential expressed genes and samples. Each row corresponds to the normalized expression value of a gene. One can obtain that there are gene clusters. THe genes that have similar expression patterns tend to be close to each other due to the hierarchical clustering. Here the interpretation possibilites are limited, since one expects that technical replicates should have more or less the same expression pattern.

### Principal component analysis
PCA (principal component analysis) plot for the samples.
<img src="assets/salmon_GRCh38_samples_pca_plot.png" style="height:500px"/>

The PCA analysis tries to capture the variance (find a projection from N dim to 2 dim that captures as much variance as possible.) One should look into the factors as well, to see which genes drive the PC1 and PC2 the most (biplot PCA). UMAP is a good alternative visualization and analysis technique. (In this mini problem it would be an overkill and misleading.)


### Fold-change versus normalized mean counts
<img src="assets/salmon_GRCh38_samples_ma_plot.png" style="height:500px"/>

MA plots are more like a quality control plot. It gives an overview about the obtained fold changes and mean counts (expression). Tipycally, genes with lower expressions have high variance, which might lead to false positive associations. It also informs us about systematic biases (irregular shapes etc). The DESeq2 carried out a normalization to 'downweight' genes with low expression values by reducing their fold change. This also helps us to guess or adjust the cut-off scores (i.e. for fold change). 
Ideally, we are interested in 'highly' exprressed genes with high log2 fold-change value (less likely to be false positive hits). For more detailed description about the applied normalization see the DESeq2 paper.

#### Vulcano plot
<img src="assets/salmon_GRCh38_samples_vulcano_plot.png" style="height:500px"/>
Vulcano plot is a scatter plot, where on y axes are the log p-values and the x axis is the fold change. This is one of the most important outputs of the analysis since it reveals the top significantly changed genes. Those genes with high fold change (positive or negative assocciation with the treatment) and 'high' significance (in that case the expression level is indirectly incorporated into the significance) are the most interesting ones. 
Here I applied a minimal statistical model JQ1 vs DMSO to compare the gene expression levels. The p-values were estimated with Wald statistics and were adjusted for multiple comparision. For the exact methods and parameters please see the attached notebooks.







### Enrichment analysis
Here I am going to evaluate the top differentially expressed genes by mapping them to gene ontology categories and KEGG pathways. For the gene ontology analaysis I use 'traditional' hypergeometric test for assessing the enrichment to GO categories. 
Another alternative approach is the gene set enrichment analysis, which takes into account all genes, not only the siginifant ones. Here I just use the KEGG pathways as examples, but there are many more classes of predefined gene sets (like cancer driver genes, hallmark genes, etc) 

### Gene ontology associations
<img src="assets/salmon_GRCh38_encrichment_go.png" style="width:1000px"/>
The main gene ontology terms that are associated with the treatments. These terms are related to replications and chromosome stability and rearrangement.

### Top5 gene ontology category and the top DE genes
<img src="assets/salmon_GRCh38_encrichment_go_gene.png" style="width:1000"/>


### Gene ontology network
<img src="assets/salmon_GRCh38_encrichment_go_network.png" style="width:1000"/>
These plots show the relations between the terms. The clustering of the terms is based on the gene ontology. Only the top40 terms were considered. One can obtain that the terms are well clustered.



# Summary


In this small study the transcription profiles of JQ1 were calculated and identified with bioinformatics techniques. JQ1 is a BET bromodaim inhibitor and a good candidate as anti-cancer agent. BET family members influence the cell-cycle progression [11]. The above expression profiles and enrichment analysis is aligned with this information (chromosome organization related terms). 
As a subsequent task I would reanalyze not only these two samples but other ones as well. The original study is a small time-course study with available time points at 3, 12, 24 hours after the treatments. Emergence of resistance to cancer therapy is one of the most challenging issues. The project data also contains resistance cell-line transcriptomic information as well (The data is available under the accession [GSE63582](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE63582)). 
The other research question is whether the new CHM13 reference genome provides new information and insights about the JQ1 treatments.


# References
